In [1]:
function mutation2(generation,AdjList, probMutation, n,k)
    newGeneration=[]
    for (i,chromosome) in enumerate(generation)
        newChromosome=chromosome[1]
        # change aleatoir. couleur de sommet 
        for u in chromosome
            p=rand()
            if p<probMutation
                available_colors = setdiff(1:k, [newChromosome[n] for n in AdjList[u]])
                if !isempty(available_colors)
                    # If there are available colors, choose one randomly
                    newChromosome[u] = rand(available_colors)
                else
                    # If no available colors, revert to a random valid color
                    newChromosome[u] = rand(1:k)
                end
            
            end
        end
        # verifie, que ca nous donne qqch bon
        for u=1:n
            needChange=false
            for v in AdjList[u]
                if mutatedChromosome[v]==mutatedChromosome[u]
                    needChange=true
                    break;
                end
            end
            if needChange
                mutatedChromosome[u]=availableColor(AdjList,mutatedChromosome,k,u)
            end
             
        end
        cnflct=countConflict(AdjList,newChromosome,n)
        push!(newGeneration,(newChromosome,cnflct))
    end

    return newGeneration;
end

mutation2 (generic function with 1 method)

In [2]:
function trouver_indice(r, p)
    somme_cumulative = 0.0
    n = length(p)
    
    for i in 1:n
        somme_cumulative += p[i]
        if r <= somme_cumulative
            return i
        end
    end
    
    # Si r dépasse toutes les sommes cumulatives, retourne -1 ou une valeur indiquant que rien n'a été trouvé.
    return -1
end

function roueFortune(generation, n, probCrossover,enfantsSize)
    parents=[]
    F=sum(pair[2] for pair in generation)
    probaGeneration=Float64[(F-pair[2])/((n-1)*F) for pair in generation]

    for i in 1:enfantsSize
        r=rand()
        index=trouver_indice(r,probaGeneration)
        push!(parents,generation[index])
    end
    return parents
end

roueFortune (generic function with 1 method)

In [3]:
using Pkg
using LinearAlgebra
using Random
using Plots
using Dates

function readGraph(file)

    #Lecture du fichier et creation structure de read_graph
    fileRead=open(file,"r")
    lines=readlines(fileRead)
    AdjList=Dict{Int,Vector{Int}}()
    n=0;
    for line in lines
        str=split(line," ")
        if(str[1]=="e")
            u=parse(Int,str[2])
            v=parse(Int,str[3])
            if haskey(AdjList,u)
                push!(AdjList[u],v)
            else
                AdjList[u]=[v]
            end

            if haskey(AdjList,v)
                push!(AdjList[v],u)
            else
                AdjList[v]=[u]
            end
        end
        if(str[1]=="p")
            n=parse(Int,str[3])
        end
    end
        return AdjList,n

end    

function randomColoring(n,k,seed=0)

    Random.seed!(seed)
    return rand(1:k,n)
end

function availableColor(adjecenceList,Colors,k,u)
    nbUsesColor=zeros(Int,k)
    for v in adjecenceList[u]
        if Colors[v]!=0
            nbUsesColor[Colors[v]]+=1
        end
    end
    for color in 1:k
        if(nbUsesColor[color]==0)
            return color
        end
    end
    return argmin(nbUsesColor)

    
end

function glouton(adjecenceList,sommetSortDegree,n,k)
    Colors=zeros(Int,n)
    for i in sommetSortDegree
        Colors[i]=availableColor(adjecenceList,Colors, k ,i)
    end
    return Colors
end

function countConflict(adjecenceList, Colors, n)
    count=0;
    for u in 1:n
        for v in adjecenceList[u]
            if(Colors[u]==Colors[v])
                count+=1
            end
        end
    end
    return count/2; 
end

function tournament(Parents,tournamentSize,popSize)
    """
        Faire tournement de taille tournamentSize
    """
    indexCandidats=rand(1:popSize,tournamentSize)
    candidats=Parents[indexCandidats]
    sort!(candidats, by=x->x[2])
    return candidats[1]
end

function crossover(Parents,AdjList, n)
    enfants=[]
    
    while true
        if length(Parents)==0
            break
        end

        if length(Parents)==1
            push!(enfants,Parents[1])
            break
        end
        
        indexA=rand(1:length(Parents))
        parentA=Parents[indexA][1]
        deleteat!(Parents,indexA)
        indexB=rand(1:length(Parents))
        parentB=Parents[indexB][1]
        deleteat!(Parents,indexB)

        seuil=rand(1:(length(parentA)-1))
        child1=parentA[1:seuil]
        append!(child1,parentB[seuil+1:end])
        child2=parentB[1:seuil];
        append!(child2,parentA[seuil+1:end])

        push!(enfants,(child1,countConflict(AdjList,child1,n)))
        push!(enfants,(child2,countConflict(AdjList,child2,n)))
    end 
    return enfants
    
end

function mutation(generation,AdjList,probMutation,n,k)
    newGeneration=[]
    for (i,chromosome) in enumerate(generation)
        mutatedChromosome=chromosome[1]
        for u=1:n
            if rand()<probMutation
                needChange=false
                for v in AdjList[u]
                    if(mutatedChromosome[v]==mutatedChromosome[u])
                        needChange=true
                        break;
                    end
                end
                if needChange
                    mutatedChromosome[u]=availableColor(AdjList,mutatedChromosome,k,u)
                end
            end  
        end
        cnflct=countConflict(AdjList,mutatedChromosome,n)
        push!(newGeneration,(mutatedChromosome,cnflct))
    end
    return newGeneration
end



function Genetic(popSize, nbGenerationMax, probMutation, tauxElitisme,tournamentSize, AdjList,degreeSort,n,k)
    startTime=now()
    Random.seed!(42)

    #plot data
    fitnessBest=[]
    fitnessAvg=[]

    # Generation initial
    Generation=[]
    solGlouton=glouton(AdjList,degreeSort,n,k)
    cnfl=countConflict(AdjList,solGlouton,n)
    push!(Generation,(solGlouton,cnfl))
    bestSolution=[];
    cnflMin=cnfl;
    for i in 2:popSize
        coloring=randomColoring(n,k,i)
        cnfl=countConflict(AdjList,coloring,n)
        if cnfl<cnflMin
            bestSolution=coloring
            cnflMin=cnfl;
        end
        push!(Generation, (coloring,cnfl));
    end

    push!(fitnessAvg,sum(pair[2] for pair in Generation)/popSize)
    push!(fitnessBest,cnflMin)

    executionTime=now()-startTime;
    
    nbGeneration=1;
    generationStatic=0
    while Dates.value(executionTime)/6000<20&&generationStatic<nbGenerationMax #temps d'execution< 5 minutes
        #Selection 
        
        selectedParents=[]
        #Tournament Selection
        
        # for i in 1:floor(tauxElitisme*popSize)
        #     push!(selectedParents,tournament(Generation,tournamentSize,popSize))
        # end

        #Roue de la Fortune Selection
        
        selectedParents=roueFortune(Generation,popSize,tauxElitisme,ceil(tauxElitisme*popSize))

        #crossover
        newGeneration=crossover(selectedParents,AdjList,n)

        #Mutation
        newGeneration=mutation(newGeneration,AdjList,probMutation,n,k)

        #new Generation
        sort!(Generation,by=x->x[2])
        if Generation[1][2]==0
            break
        end
        append!(newGeneration,Generation[1:popSize-length(newGeneration)])
        Generation=newGeneration

        push!(fitnessAvg,sum([part[2] for part in Generation])/popSize)
        sort!(Generation,by=x->x[2])
        if Generation[1][2]<cnflMin
            cnflMin=Generation[1][2]
            bestSolution=Generation[1][1]
            generationStatic=0
        else
            if Generation[1][2]==cnflMin
                generationStatic+=1
            end
        end

        push!(fitnessBest,cnflMin)
        nbGeneration+=1
    end

    sort!(Generation,by=x->x[2])
    bestSolution=Generation[1][1]
    return bestSolution,fitnessBest,fitnessAvg,nbGeneration;

end


Genetic (generic function with 1 method)

In [4]:
function count_conflict(adjacence_list, Color, n)
    """
    Compte le nombre de conflits dans une solution
    """
    Count=0
    list_conflit=[] #Liste des sommets en conflit
    bool_list_conflit=[0 for i in 1:n] # 1 si le sommet est en conflit, 0 sinon 
    for i in 1:n
        for j in adjacence_list[i]
            if Color[i]==Color[j] && j>i
                Count+=1
                if bool_list_conflit[i]==0
                    bool_list_conflit[i]=1
                    push!(list_conflit,i)
                end
                if bool_list_conflit[j]==0
                    bool_list_conflit[j]=1
                    push!(list_conflit,j)
                end
            end
        end
    end
    return Count, list_conflit
end

count_conflict (generic function with 1 method)

In [5]:
function initialize_T_conf(adjacence_list, sol, n, k)
    T_conf=zeros(Int, n, k)
    for i in 1:n
        for j in adjacence_list[i]
            T_conf[j, sol[i]]+=1
        end
    end
    return T_conf
end

function initialize_T_conf_bis(adjacence_list, sol, degree, n, k)
    T_conf=zeros(Float64, n, k)
    for i in 1:n
        for j in adjacence_list[i]
            T_conf[j, sol[i]]+=1
        end
    end
    return T_conf
end

function best_voisinbis(adjacence_list, sol, T_conf, indice_degree, n, k)
    index=0
    c_index=0
    c_index_1=0
    max=-n
    for i in indice_degree
        for c in 1:k
            if T_conf[i, sol[i]]-T_conf[i, c]>max && c!=sol[i]
                index=i
                c_index_1=sol[i]
                c_index=c
                max = T_conf[i, sol[i]]-T_conf[i, c]
            end
        end
    end
    for j in adjacence_list[index]
        T_conf[j, c_index_1]-=1
        T_conf[j, c_index]+=1
    end
    return index, c_index, max
end


function amelioration_locale(sol, adjacence_list, indice_degree, degree, n, k)
    best_sol=deepcopy(sol)
    flag=true
    step=0
    T_conf=initialize_T_conf(adjacence_list, best_sol, n, k)
    while flag && step < 200
        index, c_index, diff = best_voisinbis(adjacence_list, best_sol, T_conf, indice_degree, n, k)
        if diff >= 1 
            best_sol[index]=c_index
        else
            flag = false
        end
        step+=1
    end
    return best_sol
end

amelioration_locale (generic function with 1 method)

In [6]:
function best_voisin_tabou(adjacence_list, sol, T_conf, tabou_matrix, indice_degree, degree, step, n, k, val_act, val_min, Frequency)
    index=0
    c_index=0
    c_index_1=0
    max=-n
    diff=-n
    for i in shuffle(indice_degree)[1:100]
        explo= -1*sqrt(log(step+1)/(Frequency[i]+1))
        for c in 1:k
            if tabou_matrix[i, c]<=step || T_conf[i, sol[i]]-T_conf[i, c]>= 0.8*(val_act-val_min)+1
                if T_conf[i, sol[i]]-T_conf[i, c]- explo>max && c!=sol[i]
                    index=i
                    c_index_1=sol[i]
                    c_index=c
                    diff = T_conf[i, sol[i]]-T_conf[i, c]
                    max = T_conf[i, sol[i]]-T_conf[i, c] - explo
                end
            end
        end
    end
    if index==0
        print(step, "ERROR")
    end
    for j in adjacence_list[index]
        T_conf[j, c_index_1]-= 1
        T_conf[j, c_index]+=1
    end
    return index, c_index, diff
end

best_voisin_tabou (generic function with 1 method)

In [23]:
function tabou(adjacence_list, indice_degree, degree, n, k, sol_init)
    sol= deepcopy(sol_init)
    best_sol=deepcopy(sol_init)
    step=0
    T_conf=initialize_T_conf_bis(adjacence_list, best_sol, degree, n, k)
    Count_init = sum([T_conf[i, sol[i]] for i in 1:n])/2
    Count=[Count_init]
    min_count=Count_init
    length_tabou=1
    tabou_matrix=zeros(Int, n,k)
    cycle=Dict{Tuple, Tuple}()
    for i in 1:n
        for c in 1:k
            cycle[i,c]=(i,c)
        end
    end
    last_change=(1,1)
    last_change_tabou=0
    Tabou_length=[1]
    Frequency = zeros(Int, n)
    time_limit = 2.0
    start_time = time()
    while time() - start_time < time_limit
        if Random.rand(1:500)==1
            tabou_matrix=zeros(Int, n,k)
            length_tabou=1
        end
        index, c_index, diff = best_voisin_tabou(adjacence_list, sol, T_conf, tabou_matrix, indice_degree, degree, step, n, k, last(Count), min_count, Frequency)
        sol[index]=c_index
        if cycle[last_change]==(index, c_index)
            if length_tabou <n*k/6 && step > last_change_tabou+length_tabou
                length_tabou+=k
                last_change_tabou=step
            end
        end
        tabou_matrix[index, c_index]=step+length_tabou
        Count_step=last(Count)-diff
        if Count_step < 0
            print("Négatif",diff)
            return sol, Count, Tabou_length
        end
        push!(Count, Count_step)
        if Count_step < min_count
            min_count=Count_step
            best_sol=deepcopy(sol)
            length_tabou=1
            tabou_matrix=zeros(Int, n,k)
            last_change_tabou=step
            print(min_count, ' ', step, '\n')
            if min_count==0
                return best_sol, Count, Tabou_length
            end
        end
        cycle[last_change]=(index, c_index)
        last_change=(index,c_index)
        step+=1
        push!(Tabou_length, length_tabou)
        Frequency[index]+=1
    end
    return best_sol, Count, Tabou_length
end

tabou (generic function with 1 method)

In [38]:
#Genetic first , tabou second
function GeneticTabou(popSize,nbGenerationMax,probMutation,tauxElitisme,tournamentSize,AdjList,degree, sommetSort,n,k)
    startTime=now()
    Random.seed!(42)

    #plot data
    fitnessBest=[]
    fitnessAvg=[]

    # Generation initial
    Generation=[]
    solGlouton=glouton(AdjList,sommetSort,n,k)
    cnfl=countConflict(AdjList,solGlouton,n)
    push!(Generation,(solGlouton,cnfl))
    bestSolution=[];
    cnflMin=cnfl;
    for i in 2:popSize
        coloring=randomColoring(n,k,i)
        cnfl=countConflict(AdjList,coloring,n)
        if cnfl<cnflMin
            bestSolution=coloring
            cnflMin=cnfl;
        end
        push!(Generation, (coloring,cnfl));
    end

    push!(fitnessAvg,sum(pair[2] for pair in Generation)/popSize)
    push!(fitnessBest,cnflMin)

    executionTime=now()-startTime;
    
    nbGeneration=1;
    generationStatic=0
    while Dates.value(executionTime)/6000<20&&generationStatic<nbGenerationMax #temps d'execution< 5 minutes
        #Selection 
        
        selectedParents=[]
        #Tournament Selection
        
        # for i in 1:floor(tauxElitisme*popSize)
        #     push!(selectedParents,tournament(Generation,tournamentSize,popSize))
        # end

        #Roue de la Fortune Selection
        
        selectedParents=roueFortune(Generation,popSize,tauxElitisme,ceil(tauxElitisme*popSize))

        #crossover
        newGeneration=crossover(selectedParents,AdjList,n)

        #Mutation
        newGeneration=mutation(newGeneration,AdjList,probMutation,n,k)

        #new Generation
        sort!(Generation,by=x->x[2])
        if Generation[1][2]==0
            break
        end
        append!(newGeneration,Generation[1:popSize-length(newGeneration)])
        Generation=newGeneration

        push!(fitnessAvg,sum([part[2] for part in Generation])/popSize)
        sort!(Generation,by=x->x[2])
        if Generation[1][2]<cnflMin
            cnflMin=Generation[1][2]
            bestSolution=Generation[1][1]
            generationStatic=0
        else
            if Generation[1][2]==cnflMin
                generationStatic+=1
            end
        end
        
        push!(fitnessBest,cnflMin)
        nbGeneration+=1
    end

    sort!(Generation,by=x->x[2])
    bestSolution=Generation[1][1]
    
    finalSolution=tabou(AdjList, sommetSort, degree, n, k, bestSolution)
    return finalSolution;

end


GeneticTabou (generic function with 1 method)

In [27]:
function TabouGenetic(popSize,nbGenerationMax,probMutation,tauxElitisme,tournamentSize,AdjList,degree, sommetSort,n,k)
    startTime=now()
    Random.seed!(42)

    #plot data
    fitnessBest=[]
    fitnessAvg=[]

    # Generation initial
    Generation=[]
    solGlouton=glouton(AdjList,sommetSort,n,k)
    cnfl=countConflict(AdjList,solGlouton,n)
    push!(Generation,(solGlouton,cnfl))
    bestSolution=[];
    cnflMin=cnfl;
    for i in 2:popSize
        coloring=randomColoring(n,k,i)
        step=0
        while step <=2
            coloring=tabou(AdjList, sommetSort, degree, n, k, coloring)[1]
            step+=1
        end

        cnfl=countConflict(AdjList,coloring,n)
        if cnfl<cnflMin
            bestSolution=coloring
            cnflMin=cnfl;
        end
        push!(Generation, (coloring,cnfl));
    end

    push!(fitnessAvg,sum(pair[2] for pair in Generation)/popSize)
    push!(fitnessBest,cnflMin)

    executionTime=now()-startTime;
    
    nbGeneration=1;
    generationStatic=0
    while Dates.value(executionTime)/6000<20&&generationStatic<nbGenerationMax #temps d'execution< 5 minutes
        #Selection 
        
        selectedParents=[]
        #Tournament Selection
        
        # for i in 1:floor(tauxElitisme*popSize)
        #     push!(selectedParents,tournament(Generation,tournamentSize,popSize))
        # end

        #Roue de la Fortune Selection
        
        selectedParents=roueFortune(Generation,popSize,tauxElitisme,ceil(tauxElitisme*popSize))

        #crossover
        newGeneration=crossover(selectedParents,AdjList,n)

        #Mutation
        newGeneration=mutation(newGeneration,AdjList,probMutation,n,k)

        #new Generation
        sort!(Generation,by=x->x[2])
        if Generation[1][2]==0
            break
        end
        append!(newGeneration,Generation[1:popSize-length(newGeneration)])
        Generation=newGeneration

        push!(fitnessAvg,sum([part[2] for part in Generation])/popSize)
        sort!(Generation,by=x->x[2])
        if Generation[1][2]<cnflMin
            cnflMin=Generation[1][2]
            bestSolution=Generation[1][1]
            generationStatic=0
        else
            if Generation[1][2]==cnflMin
                generationStatic+=1
            end
        end

        push!(fitnessBest,cnflMin)
        nbGeneration+=1
    end

    sort!(Generation,by=x->x[2])
    bestSolution=Generation[1][1]
    return bestSolution,fitnessBest,fitnessAvg,nbGeneration;

end


TabouGenetic (generic function with 1 method)

In [24]:

file="dsjc1000_5.txt"
k=85
tournamentSize=2
tauxElitisme=0.1
probMutation=0.1
nbGenerationMax=1000
popSize=20

AdjList,n=readGraph(file)

#Degree de chaque sommet
degree=zeros(Int,n)
for i in 1:n
    degree[i]=size(AdjList[i])[1]
end

sommetsSort=sortperm(degree,rev=true)
gloutonSolution=glouton(AdjList,sommetsSort,n,k)

println("Glouton solution: ",countConflict(AdjList,gloutonSolution,n))

geneticColoring, fitBest,fitAvg, nbGen=Genetic(popSize,nbGenerationMax,probMutation,tauxElitisme,tournamentSize,AdjList,sommetsSort,n,k)
println(countConflict(AdjList,geneticColoring,n))


#plot([1:nbGeneration],fitnessAvg,xlabel="nb Generations",ylabel="Avg fitness in generation",title="nbGeneration vs AVGFitness")

Glouton solution: 378.0


241.0

In [29]:
k=85
tournamentSize=2
tauxElitisme=0.1
probMutation=0.1
nbGenerationMax=1000
popSize=10

#Tabou first, Genetic - second
tabouGeneticColoring, fitBest,fitAvg, nbGen=TabouGenetic(popSize,nbGenerationMax,probMutation,tauxElitisme,tournamentSize,AdjList,degree,sommetsSort,n,k)

2837.0 0
2828.0 1
2817.0 2
2808.0 3
2798.0 4
2789.0 5
2780.0 6
2769.0 7
2759.0 8
2749.0 9
2740.0 10
2730.0 11
2721.0 12
2711.0 13
2701.0 14
2692.0 15
2684.0 16
2675.0 17
2665.0 18
2656.0 19
2647.0 20
2638.0 21
2630.0 22
2621.0 23
2613.0 24
2603.0 25
2594.0 26
2585.0 27
2577.0 28
2568.0 29
2559.0 30
2551.0 31
2543.0 32
2535.0 33
2527.0 34
2518.0 35
2510.0 36
2502.0 37
2494.0 38
2486.0 39
2478.0 40
2470.0 41
2462.0 42
2454.0 43
2446.0 44
2437.0 45
2429.0 46
2421.0 47
2413.0 48
2405.0 49
2397.0 50
2390.0 51
2383.0 52
2376.0 53
2369.0 54
2361.0 55
2354.0 56
2346.0 57
2339.0 58
2332.0 59
2325.0 60
2318.0 61
2311.0 62
2304.0 63
2298.0 64
2291.0 65
2284.0 66
2277.0 67
2271.0 68
2265.0 69
2259.0 70
2252.0 71
2246.0 72
2240.0 73
2234.0 74
2226.0 75
2220.0 76
2213.0 77
2207.0 78
2201.0 79
2195.0 80
2189.0 81
2183.0 82
2176.0 83
2169.0 84
2162.0 85
2156.0 86
2150.0 87
2144.0 88
2137.0 89
2131.0 90
2124.0 91
2118.0 92
2112.0 93
2106.0 94
2100.0 95
2093.0 96
2088.0 97
2082.0 98
2076.0 99
2070.0 100


1967.0 117
1962.0 118
1956.0 119
1950.0 120
1945.0 121
1939.0 122
1934.0 123
1929.0 124
1924.0 125
1919.0 126
1913.0 127
1907.0 128
1901.0 129
1896.0 130
1891.0 131
1886.0 132
1880.0 133
1875.0 134
1869.0 135
1864.0 136
1859.0 137
1853.0 138
1847.0 139
1842.0 140
1836.0 141
1831.0 142
1826.0 143
1820.0 144
1813.0 145
1808.0 146
1803.0 147
1798.0 148
1793.0 149
1788.0 150
1783.0 151
1777.0 152
1772.0 153
1767.0 154
1762.0 155
1757.0 156
1752.0 157
1747.0 158
1742.0 159
1737.0 160
1732.0 161
1727.0 162
1722.0 163
1717.0 164
1712.0 165
1707.0 166
1702.0 167
1697.0 168
1692.0 169
1687.0 170
1682.0 171
1678.0 172
1673.0 173
1668.0 174
1663.0 175
1658.0 176
1653.0 177
1648.0 178
1643.0 179
1638.0 180
1633.0 181
1628.0 182
1623.0 183
1618.0 184
1613.0 185
1609.0 186
1604.0 187
1600.0 188
1595.0 189
1591.0 190
1586.0 191
1582.0 192
1578.0 193
1573.0 194
1569.0 195
1565.0 196
1561.0 197
1557.0 198
1552.0 199
1548.0 200
1544.0 201
1540.0 202
1536.0 203
1532.0 204
1528.0 205
1524.0 206
1520.0 20

1307.0 258
1303.0 259
1300.0 260
1297.0 261
1294.0 262
1290.0 263
1286.0 264
1282.0 265
1279.0 266
1275.0 267
1271.0 268
1268.0 269
1264.0 270
1260.0 271
1256.0 272
1252.0 273
1247.0 274
1243.0 275
1239.0 276
1236.0 277
1232.0 278
1228.0 279
1225.0 280
1222.0 281
1218.0 282
1214.0 283
1210.0 284
1207.0 285
1204.0 286
1200.0 287
1196.0 288
1192.0 289
1189.0 290
1186.0 291
1183.0 292
1180.0 293
1177.0 294
1173.0 295
1169.0 296
1166.0 297
1162.0 298
1159.0 299
1156.0 300
1152.0 301
1149.0 302
1146.0 303
1142.0 304
1138.0 305
1134.0 306
1130.0 307
1127.0 308
1124.0 309
1121.0 310
1117.0 311
1114.0 312
1111.0 313
1108.0 314
1105.0 315
1102.0 316
1099.0 317
1095.0 318
1092.0 319
1089.0 320
1086.0 321
1083.0 322
1079.0 323
1076.0 324
1073.0 325
1070.0 326
1067.0 327
1064.0 328
1060.0 329
1057.0 330
1053.0 331
1050.0 332
1047.0 333
1044.0 334
1041.0 335
1037.0 336
1034.0 337
1031.0 338
1027.0 339
1024.0 340
1021.0 341
1018.0 342
1014.0 343
1011.0 344
1008.0 345
1005.0 346
1002.0 347
999.0 348


892.0 386
889.0 387
887.0 388
885.0 389
882.0 390
879.0 391
876.0 392
873.0 393
870.0 394
868.0 395
865.0 396
862.0 397
860.0 398
857.0 399
854.0 400
852.0 401
850.0 402
847.0 403
845.0 404
843.0 405
841.0 406
839.0 407
837.0 408
834.0 409
832.0 410
829.0 411
827.0 412
824.0 413
822.0 414
820.0 415
818.0 416
816.0 417
814.0 418
812.0 419
810.0 420
808.0 421
806.0 422
804.0 423
802.0 424
799.0 425
796.0 426
793.0 427
791.0 428
789.0 429
786.0 430
784.0 431
782.0 432
780.0 433
778.0 434
775.0 435
773.0 436
770.0 437
768.0 438
766.0 439
764.0 440
761.0 441
759.0 442
757.0 443
755.0 444
753.0 445
751.0 446
749.0 447
747.0 448
745.0 449
743.0 450
741.0 451
739.0 452
737.0 453
735.0 454
733.0 455
731.0 456
729.0 457
726.0 458
724.0 459
722.0 460
720.0 461
718.0 462
715.0 463
713.0 464
711.0 465
710.0 466
708.0 467
707.0 468
705.0 469
704.0 470
702.0 471
700.0 472
698.0 473
696.0 474
695.0 475
693.0 476
691.0 477
689.0 478
687.0 479
686.0 480
685.0 481
683.0 482
681.0 483
679.0 484
678.0 485


603.0 532
602.0 533
601.0 534
599.0 535
597.0 536
595.0 537
594.0 538
592.0 539
591.0 540
589.0 541
588.0 542
587.0 543
586.0 544
584.0 545
583.0 546
582.0 547
581.0 548
580.0 549
578.0 550
576.0 551
575.0 552
574.0 553
572.0 554
571.0 555
570.0 556
569.0 557
568.0 558
566.0 559
565.0 560
564.0 561
562.0 562
561.0 563
559.0 564
558.0 565
557.0 566
556.0 567
555.0 568
554.0 569
552.0 570
550.0 571
549.0 572
548.0 573
547.0 574
546.0 575
544.0 576
543.0 577
542.0 578
541.0 579
540.0 580
539.0 581
538.0 582
537.0 583
536.0 584
535.0 585
534.0 586
533.0 587
532.0 588
530.0 589
529.0 590
528.0 591
527.0 592
526.0 593
525.0 594
524.0 595
522.0 596
521.0 597
520.0 598
519.0 599
518.0 600
517.0 601
516.0 602
514.0 603
512.0 604
510.0 605
509.0 606
508.0 607
507.0 608
506.0 609
505.0 610
503.0 611
502.0 612
501.0 613
500.0 614
499.0 615
498.0 616
497.0 617
496.0 618
495.0 619
494.0 620
493.0 621
492.0 622
491.0 623
490.0 624
489.0 625
488.0 626
487.0 627
486.0 628
485.0 629
484.0 630
482.0 631


...

76.0 20095


75.0 22378
74.0 23570


73.0 24331
72.0 24436
71.0 24792


70.0 26592


([12, 62, 77, 39, 4, 85, 83, 5, 8, 37  …  68, 19, 62, 1, 44, 20, 48, 21, 79, 11], Any[61.0, 61.0, 61.0, 61.0, 61.0, 61.0, 61.0, 61.0, 61.0, 61.0  …  61.0, 61.0, 61.0, 61.0, 61.0, 61.0, 61.0, 61.0, 61.0, 61.0], Any[98.0, 67.4, 66.6, 65.8, 65.8, 65.4, 64.8, 64.8, 64.7, 64.7  …  61.0, 61.0, 61.0, 61.0, 61.0, 61.0, 61.0, 61.0, 61.0, 61.0], 1001)

In [33]:
countConflict(AdjList,geneticTabouColoring,n)

61.0

In [39]:
k=85
tournamentSize=2
tauxElitisme=0.1
probMutation=0.1
nbGenerationMax=1000
popSize=10

#Tabou first, Genetic - second
geneticTabouColoring=GeneticTabou(popSize,nbGenerationMax,probMutation,tauxElitisme,tournamentSize,AdjList,degree,sommetsSort,n,k)

251.0 8
250.0 11
249.0 15
248.0 23
247.0 28
246.0 29
245.0 36
244.0 40
243.0 52
242.0 57
241.0 83
240.0 87
239.0 90
238.0 100
237.0 103
236.0 109
235.0 117
234.0 130
233.0 132
232.0 142
231.0 148
230.0 153
229.0 158
228.0 187
227.0 192
226.0 206
225.0 211
224.0 214
223.0 218
222.0 241
221.0 272
220.0 277
219.0 289
218.0 304
217.0 321
216.0 327
215.0 344
214.0 361
213.0 363
212.0 386
211.0 388
210.0 390
209.0 396
208.0 408
207.0 415
206.0 420
205.0 444
204.0 468
203.0 472
202.0 480
201.0 481
200.0 483
199.0 488
198.0 489
197.0 490
196.0 494
195.0 495
194.0 505
193.0 529
192.0 530
191.0 539
190.0 566
189.0 567
188.0 574


187.0 633
186.0 705
185.0 728
184.0 733
183.0 1140
182.0 1141
181.0 1300
180.0 1343
179.0 1407
178.0 1408
177.0 1544
176.0 1553
175.0 1615
174.0 1618
173.0 1619
172.0 1657
171.0 1659
170.0 1661
169.0 1675
168.0 1792
167.0 1793


166.0 1916
165.0 2007
164.0 2013
163.0 2043
162.0 2044
161.0 2113
160.0 2115
159.0 2118
158.0 2121
157.0 2127
156.0 2129
155.0 2133
154.0 2156
153.0 2188
152.0 2222
151.0 2261
150.0 2266
149.0 2294
148.0 2407
147.0 2420
146.0 2439
145.0 2499
144.0 2584
143.0 2616
142.0 2622
141.0 2629
140.0 2646
139.0 2738
138.0 2760
137.0 2780


136.0 3234
135.0 3237
134.0 3515
133.0 3537
132.0 3551
131.0 3554
130.0 3671
129.0 3674
128.0 3711
127.0 3733
126.0 3788
125.0 3954
124.0 4008
123.0 4034
122.0 4145
121.0 4275
120.0 4278
119.0 4326


118.0 4595
117.0 4613
116.0 4763
115.0 4851
114.0 4906
113.0 4908
112.0 5045
111.0 

...

96.0 14069
95.0 14091
94.0 14405
93.0 14738
92.0 14744
91.0 14972
90.0 15415


89.0 16254


88.0 20343


87.0 25076
86.0 25150


([64, 36, 28, 67, 50, 26, 57, 14, 59, 26  …  51, 70, 37, 75, 67, 22, 66, 22, 31, 83], [252.0, 252.0, 252.0, 252.0, 252.0, 252.0, 252.0, 252.0, 252.0, 251.0  …  87.0, 87.0, 87.0, 87.0, 87.0, 87.0, 87.0, 87.0, 87.0, 87.0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  86, 86, 86, 86, 86, 86, 86, 86, 86, 86])

In [42]:
println(countConflict(AdjList,geneticGenTabouColoring[1],n))

86.0
